# import necessary libraries

In [1]:
!git clone https://github.com/vedpd/USD-MS-AAI.git
%cd USD-MS-AAI/Module\ 2
!git checkout module2-bharath

Cloning into 'USD-MS-AAI'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 115 (delta 38), reused 62 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 11.62 MiB | 22.71 MiB/s, done.
Resolving deltas: 100% (38/38), done.
/content/USD-MS-AAI/Module 2
Branch 'module2-bharath' set up to track remote branch 'module2-bharath' from 'origin'.
Switched to a new branch 'module2-bharath'


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

os.getcwd()
os.listdir()


import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
import torch



# Read the Symptoms and Disease data

In [4]:

# Load data
Symp_Disease_data = pd.read_csv("/content/USD-MS-AAI/Module 2/Data/Train_data.csv")
Symp_Disease_data.shape



(1200, 3)

# Data Clean up

In [5]:

def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    return text.strip()

# Apply cleaning - replace 'text_column' with the actual text feature name
Symp_Disease_data['cleaned_text'] = Symp_Disease_data['text'].apply(clean_text)


In [6]:
# before cleaning
print(Symp_Disease_data['text'][0],"\n")

#after cleaning
print(Symp_Disease_data['cleaned_text'][0])

I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches. 

i have been experiencing a skin rash on my arms legs and torso for the past few weeks it is red itchy and covered in dry scaly patches


# Label Encoding of the Target variable - convert disease into numeric values

In [7]:
# Encode labels
label_encoder = LabelEncoder()
Symp_Disease_data["label_id"] = label_encoder.fit_transform(Symp_Disease_data["label"])


In [8]:
Symp_Disease_data[['label','label_id']].sort_values('label_id',ascending=True).drop_duplicates()

,label,label_id
579,Acne,0
538,Arthritis,1
945,Bronchial Asthma,2
1053,Cervical spondylosis,3
199,Chicken pox,4
377,Common Cold,5
264,Dengue,6
455,Dimorphic Hemorrhoids,7
318,Fungal infection,8
987,Hypertension,9


# Train-test split

In [9]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    Symp_Disease_data["cleaned_text"], Symp_Disease_data["label_id"], test_size=0.30, random_state=42, stratify=Symp_Disease_data["label_id"]
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.50, random_state=42, stratify=temp_labels
)


# Save vocab_size and num_classes for later use

print(f"Sample encoded text: {train_texts.shape}")
print(f"Encoded label: {train_labels.shape}")

Sample encoded text: (840,)
Encoded label: (840,)


# Tokenization

In [10]:

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=64)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=64)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
print(train_encodings)

{'input_ids': [[101, 1045, 2031, 2042, 13417, 10720, 2015, 1998, 19197, 2045, 2003, 1037, 2844, 3255, 1999, 2026, 2067, 1998, 2036, 2369, 2026, 2159, 1045, 2031, 2036, 4384, 2235, 2417, 7516, 2006, 2026, 2067, 1998, 3300, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2031, 1037, 14768, 24171, 5510, 1999, 2026, 2677, 1045, 2031, 6976, 19337, 8450, 1998, 20934, 14536, 2075, 1045, 2031, 1037, 3110, 1997, 3778, 2030, 2440, 2791, 1999, 2026, 3356, 13878, 2008, 2197, 2005, 1037, 2261, 2847, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 4921, 2063, 2042, 1055, 24045, 6774, 4297, 7971, 15706, 1998, 1045, 2074, 2064, 2102, 2131, 2023, 10720, 2000, 2175, 2185, 1045, 2514, 2061, 5410, 1998, 9069, 1998, 2026, 19340, 2180, 2102, 2644, 2026, 4860, 2003, 2428, 2152, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 4921, 2063, 2018, 6387, 2009, 8450, 10720, 2015, 24780, 1998

# Create PyTorch Dataset

In [12]:
import torch

class SymptomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {"labels": torch.tensor(self.labels[idx])}

train_dataset = SymptomDataset(train_encodings, train_labels.tolist())
val_dataset = SymptomDataset(val_encodings, val_labels.tolist())
test_dataset = SymptomDataset(test_encodings, test_labels.tolist())


# Model Definition (BERT + Classification Head)

### BertForSequenceClassification is a pre-trained BERT model fine-tuned for text classification tasks such as sentiment analysis, spam detection, or topic classification. It adds a classification head on top of the [CLS] token output from BERT's final layer.  This model is widely used due to its strong contextual understanding and high accuracy across NLP tasks.

In [13]:

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=24
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training Setup

In [14]:
# Load model & tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=24)

# Define TrainingArguments (no evaluation_strategy here)
training_args = TrainingArguments(
    output_dir="./results",
    save_steps=1000,          # save checkpoint every N steps
    save_total_limit=1,       # keep only last checkpoint
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=50,
    do_eval=True              # make sure evaluation is enabled
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # from your earlier code
    eval_dataset=val_dataset,     # from your earlier code
    tokenizer=tokenizer
)

# Train with manual evaluation each epoch
for epoch in range(int(training_args.num_train_epochs)):
    print(f"\n===== Epoch {epoch+1} / {training_args.num_train_epochs} =====")
    trainer.train()
    print("\n*** Running Evaluation ***")
    trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-567258242.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



===== Epoch 1 / 3 =====


Step,Training Loss
50,3.140900
100,2.642200
150,1.907600
200,1.370700
250,0.962200
300,0.796800



*** Running Evaluation ***



===== Epoch 2 / 3 =====


Step,Training Loss
50,0.612800
100,0.346600
150,0.197400
200,0.092800
250,0.054100
300,0.041300



*** Running Evaluation ***

===== Epoch 3 / 3 =====


Step,Training Loss
50,0.028900
100,0.019600
150,0.014000
200,0.011800
250,0.010500
300,0.010400



*** Running Evaluation ***


# Train the Model

In [15]:
trainer.train()

Step,Training Loss
50,0.009000
100,0.007500
150,0.006400
200,0.005900
250,0.005400
300,0.005400


TrainOutput(global_step=315, training_loss=0.006528898127495296, metrics={'train_runtime': 78.7291, 'train_samples_per_second': 32.009, 'train_steps_per_second': 4.001, 'total_flos': 81601098122880.0, 'train_loss': 0.006528898127495296, 'epoch': 3.0})

# Model Evaluation

In [16]:
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(axis=1)

from sklearn.metrics import classification_report

print(classification_report(test_labels, pred_labels, target_names=label_encoder.classes_))


                                 precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00         7
                      Arthritis       1.00      1.00      1.00         7
               Bronchial Asthma       1.00      1.00      1.00         7
           Cervical spondylosis       1.00      1.00      1.00         8
                    Chicken pox       1.00      0.88      0.93         8
                    Common Cold       1.00      1.00      1.00         7
                         Dengue       0.89      1.00      0.94         8
          Dimorphic Hemorrhoids       1.00      1.00      1.00         8
               Fungal infection       1.00      1.00      1.00         7
                   Hypertension       1.00      1.00      1.00         7
                       Impetigo       1.00      1.00      1.00         8
                       Jaundice       1.00      1.00      1.00         7
                        Malaria       1.00      1.

In [ ]:
# on train data

In [17]:
train_predictions = trainer.predict(train_dataset)
train_pred_labels = train_predictions.predictions.argmax(axis=1)

from sklearn.metrics import classification_report

print(classification_report(train_labels, train_pred_labels, target_names=label_encoder.classes_))



                                 precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00        35
                      Arthritis       1.00      1.00      1.00        35
               Bronchial Asthma       1.00      1.00      1.00        35
           Cervical spondylosis       1.00      1.00      1.00        35
                    Chicken pox       1.00      1.00      1.00        35
                    Common Cold       1.00      1.00      1.00        35
                         Dengue       1.00      1.00      1.00        35
          Dimorphic Hemorrhoids       1.00      1.00      1.00        35
               Fungal infection       1.00      1.00      1.00        35
                   Hypertension       1.00      1.00      1.00        35
                       Impetigo       1.00      1.00      1.00        35
                       Jaundice       1.00      1.00      1.00        35
                        Malaria       1.00      1.

# Deployment-Ready Inference Function

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # ensure model is on GPU (or CPU if no GPU)

def predict_disease(text):
    # Tokenize
    inputs = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=64,
        return_tensors="pt"
    )

    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class_id = torch.argmax(outputs.logits, dim=1).item()

    return label_encoder.classes_[predicted_class_id]

# Example
print(predict_disease("high fever, severe headache, joint pain"))

Dengue


In [ ]:
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(axis=1)

# Key metric for evluation - f1 score

In [18]:
from sklearn.metrics import f1_score

# Macro F1 = average F1 across all classes (treats all classes equally)
macro_f1 = f1_score(test_labels, pred_labels, average='macro')

# Weighted F1 = average F1 weighted by number of samples in each class
weighted_f1 = f1_score(test_labels, pred_labels, average='weighted')

print(f"Macro F1-score: {macro_f1:.4f}")
print(f"Weighted F1-score: {weighted_f1:.4f}")

Macro F1-score: 0.9891
Weighted F1-score: 0.9888
